PyEOM is Python package for post-Hartree-Fock calculations of excited states based on the equation-of-motion framework.

This tutorial is a quick introduction to PyEOM library, and introduces the following topics:
* Specifiying a molecular system (Hamiltonian and RDMs)
* Evaluating ionization potentials
* Evaluating excitation energies

<!-- * Calculating the transition density matrix for the oscillator strength -->


## Specifiying a molecular system

All high-level methods in this package require:
* the electronic Hamiltonian
* the approximated wavefunction for the reference (ground) state

The Hamiltonian must be specified through the one- and two-electron integrals and the reference ground state through its 1- and 2-electron reduced density matrices (RDMs).


In the next cell, the Hamiltonian for the Beryllium atom is loaded from Numpy arrays. Note that using [IOData](https://iodata.readthedocs.io/en/latest/) the Hamiltonian input parameters can alternatively be loaded from a FCIDUMP file generated by an external quantum chemistry package.

In [1]:
# Import statements
import numpy as np
from eomee.tools.tools import spinize
from eomee.test import find_datafile


# Load the one- and two-electron integrals in the restricted MO basis
one_int = np.load(find_datafile('be_sto3g_oneint.npy'))
two_int = np.load(find_datafile('be_sto3g_twoint.npy'))
one_sp = spinize(one_int) # (k,k) --> (2k, 2k) matrix dimensions
two_sp = spinize(two_int) # (k,k,k,k) --> (2k, 2k, 2k, 2k) array dimensions

The loaded electron integrals (`one_int` and `two_int`) are given in a restricted molecular orbital basis (as is commonly the case with post-HF approaches) while PyEOM methods are represented in terms of generalized spin-orbital. The utility function `spinize` can be used to facilitate the mapping to our internal format.

In this package, the $N$-electron ground state is defined through its one- and two-electron reduced density matrices (RDMs).
The RDMs have to be loaded from Numpy arrays. However, for single Slater determinant wavefunction models like the Hartree-Fock wavefunction, PyEOM provides the tool function `hartreefock_rdms` to generate the RDMs.

In [2]:
from eomee.tools.tools import hartreefock_rdms

# Generate the Hartree-Fock 1- and 2-RDMs
nelecs = (2, 2)
nbasis = one_int.shape[0]
rdm1, rdm2 = hartreefock_rdms(nbasis, *nelecs)

## Computing spectroscopic properties

### Ionization Potentials

To compute the ionization potentials (IP) use one of the method `IP`, `IPc` or `IPa`. An istances of these classes encodes the generalized eigenvalue problem that must be solved to get the transition energies and the associated transition operators corresponding to removing one electron from the ground state.

The next cell illustrates how to create an instance of the class `IP` and how to access some of its attributes such as the matrices for the eigenvalue problem representing this method. (For the HF reference `IP` is equivalent to the Koomans' Theorem).

In [3]:
from eomee.eomip import IP

ekt = IP(one_sp, two_sp, rdm1, rdm2)


print("Generalized Fock matrix shape:\n", ekt.lhs.shape)
print("Diagonal elements of Generalized Fock matrix:\n", np.diag(ekt.lhs))
print("Diagonal elements of metric matrix:\n", np.diag(ekt.rhs))
print("Number of eigenvalue/vectors", ekt.neigs)

Generalized Fock matrix shape:
 (10, 10)
Diagonal elements of Generalized Fock matrix:
 [ 4.48399211  0.25403769 -0.         -0.         -0.          4.48399211
  0.25403769 -0.         -0.         -0.        ]
Diagonal elements of metric matrix:
 [1. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
Number of eigenvalue/vectors 10


The full ionization spectrum can be obtained with the `solve_dense` method.

In [4]:
# Solve the EKT eigenvalue problem
ips, coeffs = ekt.solve_dense(normalize=True)

# Print the first 5 ionization energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ips[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

n  EDiff.(a.u.)
0  0.254 
1  0.254 
2  4.48  
3  4.48  


For Beryllium, with the HF density matrices, EKT reduced to Koopmans' Theorem. Therefore, the ionization potentials correspond to the energies of the occupied HF molecular orbitals; two values for the alpha and beta electrons in the 1s atomic orbital and correspondingly for the electrons in the 2s orbital.

Alternatively, only a few transitions can be determined using the `solve_sparse` method. The number of lowest excitation energies to be computed is specified by the parameter `nsols`.

In [5]:
(e1, e2, e3) = ekt.solve_sparse(nsols=3, sigma=1)[0]

print(f"Three lowest IPs for Be (a.u.):\n  {e1:.3}, {e2:.3}, {e3:.3}")

Three lowest IPs for Be (a.u.):
  4.44e-16, 0.254, 0.254


### Electronic excitation energies

To get excitation energies use the method `EE` (or the alternative `EEm` method). The istance of this class encodes the generalized eigenvalue problem that must be solved to get the excitation energies and the corresponding transition coefficients.

In [6]:
from eomee.excitation import EE

erpa = EE(one_sp, two_sp, rdm1, rdm2)

print("Orbital Hessian matrix shape:\n", erpa.lhs.shape)
print("Diagonal elements of orbital Hessian matrix:\n", np.diag(erpa.lhs))
print("Diagonal elements of metric matrix:\n", np.diag(erpa.rhs))
print("Number of eigenvalue/vectors", erpa.neigs)

Orbital Hessian matrix shape:
 (100, 100)
Diagonal elements of orbital Hessian matrix:
 [ 0.00000000e+00  2.22044605e-16  4.15144831e+00  4.15144831e+00
  4.15144831e+00 -8.88178420e-16  0.00000000e+00  4.13584923e+00
  4.13584923e+00  4.13584923e+00  0.00000000e+00  2.22044605e-16
  1.52335727e-01  1.52335727e-01  1.52335727e-01  0.00000000e+00
  0.00000000e+00  6.55267471e-02  6.55267471e-02  6.55267471e-02
  4.15144831e+00  1.52335727e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.13584923e+00  6.55267471e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  4.15144831e+00  1.52335727e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  4.13584923e+00
  6.55267471e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  4.15144831e+00  1.52335727e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.13584923e+00  6.55267471e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -8.88178420e-16  2.22044605e-16
  4.13584923e+00  4.13584923e+00  4.13584923e+00  0.00000000e+00
  

All electronic excitations of Beryllium can be obtained with the `solve_dense` method.

In [7]:
ev = erpa.solve_dense()[0]


# Print the first 10 excitation energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(ev[:10]):
    print("{0:<2} {1:<6.3}".format(i, e))

n  EDiff.(a.u.)
0  0.222 
1  0.222 
2  0.222 
3  4.14  
4  4.14  
5  4.14  
6  4.14  
7  4.14  
8  4.14  
9  4.14  


For closed-shell systems like Beryllium, one can use the spin-adapted implementations `EES` and `EET` to compute the singlet and triplet excitation energies, respectively. These EOM methods are subclasses of `EE`-EOM and therefore inherit its methods.

In [8]:
from eomee.spinadapted.excitation import EET

erpa_t = EET(one_sp, two_sp, rdm1, rdm2)
evt = erpa_t.solve_dense()[0]


# Print the triplet transition energies
print("{0:<2s} {1:<6s}".format('n', 'EDiff.(a.u.)'))
for i, e in enumerate(evt[:5]):
    print("{0:<2} {1:<6.3}".format(i, e))

n  EDiff.(a.u.)
0  4.14  
1  4.14  
2  4.14  


## Post-processing EOM results

For a transition process of interest, it is also posible to compute the transition density matrices (TDMs) with the `compute_tdm` method.

The next cell illustrates how to compute the TDM between the ground state and the lowest $(N-1)$-electron 
excited state from the eigenvectors of the `IP`-EOM equation.

In [9]:
tdm = ekt.compute_tdm(coeffs[0])

print("Tansition matrix from lowest IP eigenvector: \n", tdm)

Tansition matrix from lowest IP eigenvector: 
 [-6.56167252e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
